In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Starting H2O

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
train = h2o.import_file('../input/ncaam-2021-features-only/tourney.csv')
test = h2o.import_file('../input/ncaam-2021-features-only/test.csv')

In [ ]:
x = test.columns[4:]
y = 'result'
train[y] = train[y].asfactor()

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=10000, seed=47, max_runtime_secs=31000)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
h2o.remove(train)

In [ ]:
preds = aml.predict(test)
preds['p1'].as_data_frame().values.flatten().shape

In [ ]:
MSampleSubmission = pd.read_csv('../input/ncaam-march-mania-2021/MSampleSubmissionStage1.csv')
MSampleSubmission.head()

In [ ]:
test_preds = preds['p1'].as_data_frame().values
idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test['ID'].as_data_frame(), pd.Series(test_preds.flatten())], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = MSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub.to_csv('h2o_automl_submission_2.csv', index=False)

In [ ]:
sub.head()